#Install Library

[RDKit ](https://github.com/rdkit/rdkit)

[DGL](https://github.com/dmlc/dgl/)

[DGL-LifeSci](https://github.com/awslabs/dgl-lifesci)
[link text](https://)

In [ ]:
!pip install rdkit-pypi
!pip install dgllife
!pip install dgl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 13.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinu

# Import Library

In [ ]:
import os
# os.environ['PYTHONHASHSEED'] = '0'
# os.environ['CUDA_VISIBLE_DEVICES']='-1'
# os.environ['TF_CUDNN_USE_AUTOTUNE'] ='0'

import dgl
import sys
import torch
import random
import cv2
import torchvision
import pandas as pd
import numpy as np
import tensorflow as tf
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms

from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit import DataStructs

from torch.utils.data import DataLoader, Dataset
from dgllife.data import SIDER
from numpy import array
from numpy import argmax
from tensorflow.keras.utils import to_categorical
from dgllife.model import load_pretrained
from tensorflow.keras.callbacks import  History
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer, AttentiveFPAtomFeaturizer
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
from dgl.data.utils import get_download_dir, download, _get_dgl_url, extract_archive
from dgllife.data.csv_dataset import MoleculeCSVDataset
from dgllife.utils.mol_to_graph import smiles_to_bigraph

# random.seed(46)
# np.random.seed(46)
# tf.random.set_seed(46)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


# Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_file_path = '/content/drive/My Drive/market_approvability_train.csv'
test_file_path = '/content/drive/MyDrive/market_approvability_test.csv'

In [ ]:
df_train =  pd.read_csv(train_file_path, usecols=['smiles', 'label'])
df_test = pd.read_csv(test_file_path , usecols=['smiles', 'label'])

In [ ]:
df_train

,smiles,label
0,CN([C@H](C)c1cc(cc(c1)C(F)(F)F)C(F)(F)F)C(=O)N...,0
1,CC(C)[C@](C)(O)/C=C/[C@@H](C)[C@H]1CC[C@H]2/C(...,0
2,CN(CC1=Cc2ccccc2Oc2ccccc12)CC#C,0
3,CO/C(/O)=C1/[C@H](C(C(=O)O[C@H]2CN(Cc3ccccc3)C...,0
4,CO/N=C1\CN(C[C@H]\1CN)c1[n]c2c(cc1F)c(=O)c(c[n...,1
...,...,...
8442,O=c1[n]2C=C[C@@H]3O[C@@H]3c2[n][n]1CCCN1CCN(CC...,1
8443,NC(N)=NS(=O)(=O)c1ccc(N)cc1,1
8444,NCCCC[C@@H](NC(=O)[C@H](Cc1cc(Br)c(O)c(Br)c1)N...,0
8445,CCCN(CCC)C(=O)[C@@H](Cc1ccc(cc1)OCCN(CC)CC)NC(...,1


# Class & Functions

In [ ]:
class DATASET(MoleculeCSVDataset):

    def __init__(self,dataFram=None,
                 smiles_to_graph=smiles_to_bigraph,
                 node_featurizer=None,
                 edge_featurizer=None,
                 load=False,
                 log_every=1000,
                 cache_file_path = None,
                 n_jobs=1):


        super(DATASET, self).__init__(df=dataFram,
                                    smiles_to_graph=smiles_to_graph,
                                    node_featurizer=node_featurizer,
                                    edge_featurizer=edge_featurizer,
                                    smiles_column='smiles',
                                    cache_file_path = cache_file_path,
                                    load=load,
                                    log_every=log_every,
                                    init_mask=True,
                                    n_jobs=n_jobs)

    def __getitem__(self, item):

        return self.smiles[item], self.graphs[item], self.labels[item], self.mask[item]

In [ ]:
from dgllife.model import load_pretrained
from dgllife.model import MLPPredictor

def get_sider_model (model_name = "GCN_attentivefp_SIDER"):
    if model_name ==  "GCN_attentivefp_SIDER":
        predictor_dropout = 0.08333992387843633
        predictor_hidden_feats = 1024
    else:
        predictor_dropout =  0.034959769945995006
        predictor_hidden_feats = 512

    gcn = load_pretrained(model_name)
    gnn_out_feats = gcn.gnn.hidden_feats[-1]
    gcn.predict = MLPPredictor(2 * gnn_out_feats, predictor_hidden_feats, predictor_hidden_feats, predictor_dropout)

    return gcn

In [ ]:
def create_dataset_with_gcn(dataset, GCN ):
    created_data = []
    for i, data in enumerate(dataset):
        smiles, g, label, mask = data
        g = g.to(device)
        g = dgl.add_self_loop(g)
        graph_feats = g.ndata.pop('h')
        embbed = GCN(g, graph_feats)
        embbed = embbed.to('cpu')
        embbed = embbed.detach().numpy()
        d = (embbed, label, smiles)
        created_data.append(d)
    print('Data created!!')
    return created_data

In [ ]:
def count_lablel(dataset):

    label_pos = 0
    lable_neg = 0

    for i, data in enumerate(dataset):
        embbed, lbl, smiles = data
        if lbl == 1.:
            label_pos += 1
        else:
            lable_neg += 1

    return label_pos, lable_neg

# Classification with CEBCP and AttentiveFp GCN

In [ ]:
train_dataset = DATASET(df_train, smiles_to_bigraph, AttentiveFPAtomFeaturizer(), cache_file_path="./sider_dglgraph.bin")
test_dataset = DATASET(df_test, smiles_to_bigraph, AttentiveFPAtomFeaturizer(), cache_file_path="./sider_dglgraph.bin")

Processing dgl graphs from scratch...
Processing molecule 1000/8447
Processing molecule 2000/8447
Processing molecule 3000/8447
Processing molecule 4000/8447
Processing molecule 5000/8447
Processing molecule 6000/8447
Processing molecule 7000/8447
Processing molecule 8000/8447
Processing dgl graphs from scratch...


In [ ]:
model_name = 'GCN_attentivefp_SIDER'
gcn_model = get_sider_model(model_name)
gcn_model.eval()
gcn_model = gcn_model.to(device)

GCN_attentivefp_SIDER_pre_trained.pth:   0%|          | 0.00/3.09M [00:00<?, ?B/s]

Pretrained model loaded


In [ ]:
train_ds = create_dataset_with_gcn(train_dataset, gcn_model)
test_ds = create_dataset_with_gcn(test_dataset, gcn_model)

Data created!!
Data created!!


In [ ]:
label_pos , label_neg = count_lablel(train_ds)
print(f'train positive label: {label_pos} - train negative label: {label_neg}')

label_pos , label_neg = count_lablel(test_ds)
print(f'Test positive label: {label_pos} - Test negative label: {label_neg}')

train positive label: 5148 - train negative label: 3299
Test positive label: 558 - Test negative label: 381


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def sequential_model():
    inputs = Input(shape=(1024,))  # Define the input shape

    # Define the network
    L1 = Dense(1024, activation='relu')(inputs)
    L1_D = Dropout(0.2)(L1)

    L2 = Dense(512, activation='relu')(L1_D)
    L2_D = Dropout(0.2)(L2)

    L3 = Dense(256, activation='relu')(L2_D)
    L3_D = Dropout(0.2)(L3)

    L4 = Dense(128, activation='relu')(L3_D)
    L4_D = Dropout(0.2)(L4)

    L5 = Dense(64, activation='relu')(L4_D)
    L5_D = Dropout(0.2)(L5)

    L6 = Dense(32, activation='relu')(L5_D)
    L6_D = Dropout(0.2)(L6)

    L7 = Dense(8, activation='relu')(L6_D)

    prediction = Dense(1, activation='sigmoid')(L7)  # Output layer

    # Create the model
    sequential_net = Model(inputs=inputs, outputs=prediction)

    # Compile the model
    optimizer = Adam(learning_rate=0.001)
    sequential_net.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=["accuracy", "mae", "mse", tf.keras.metrics.AUC()])

    return sequential_net

In [ ]:
from sklearn.model_selection import KFold

Epoch_S = 10

train = []
lbls_train = []
valid = []
lbls_valid = []

for i , data in enumerate(train_ds):
    embbed, lbl, smiles = data
    train.append(embbed[0])
    lbls_train.append(lbl.tolist())

for i , data in enumerate(test_ds):
    embbed, lbl, smiles = data
    valid.append(embbed[0])
    lbls_valid.append(lbl.tolist())

train = np.array(train).reshape(-1,1024,1)
lbls_train = np.array(lbls_train)

valid = np.array(valid).reshape(-1,1024,1)
lbls_valid = np.array(lbls_valid)

# create neural network model
sequential_net = sequential_model()
history = History()
P = sequential_net.fit(train, lbls_train, epochs = Epoch_S, batch_size = 128, callbacks=[history])

for j in range(100):
    C=1
    Before = int(P.history['accuracy'][-1]*100)
    for i in range(2,Epoch_S+1):
        if  int(P.history['accuracy'][-i]*100) == Before:
            C=C+1
        else:
            C=1
        Before=int(P.history['accuracy'][-i]*100)
        print(Before)
    if C==Epoch_S:
        break
    P = sequential_net.fit(train, lbls_train, epochs = Epoch_S, batch_size = 128, callbacks=[history])
print(j+1)

score  = sequential_net.evaluate(valid, lbls_valid, verbose=1)

Epoch 1/10
66/66 [==============================] - 9s 91ms/step - loss: 0.6795 - accuracy: 0.5810 - mae: 0.4747 - mse: 0.2419 - auc: 0.5527
Epoch 2/10
66/66 [==============================] - 3s 48ms/step - loss: 0.6352 - accuracy: 0.6309 - mae: 0.4434 - mse: 0.2219 - auc: 0.6604
Epoch 3/10
66/66 [==============================] - 3s 44ms/step - loss: 0.5930 - accuracy: 0.6873 - mae: 0.4074 - mse: 0.2024 - auc: 0.7329
Epoch 4/10
66/66 [==============================] - 3s 45ms/step - loss: 0.5553 - accuracy: 0.7194 - mae: 0.3745 - mse: 0.1868 - auc: 0.7764
Epoch 5/10
66/66 [==============================] - 4s 59ms/step - loss: 0.5284 - accuracy: 0.7384 - mae: 0.3578 - mse: 0.1768 - auc: 0.8009
Epoch 6/10
66/66 [==============================] - 4s 57ms/step - loss: 0.4917 - accuracy: 0.7619 - mae: 0.3256 - mse: 0.1618 - auc: 0.8333
Epoch 7/10
66/66 [==============================] - 3s 45ms/step - loss: 0.4618 - accuracy: 0.7871 - mae: 0.3032 - mse: 0.1501 - auc: 0.8558
Epoch 8/10
66

In [ ]:
score

[1.4144773483276367,
 0.805111825466156,
 0.19343821704387665,
 0.16682250797748566,
 0.848371148109436]

In [ ]:
# from sklearn.model_selection import KFold

# Epoch_S = 10

# def evaluate_model(df, k = 10 , shuffle = False):

#     result = []
#     s = 0

#     kf = KFold(n_splits=10, shuffle= shuffle, random_state=None)

#     for train_index, test_index in kf.split(df):

#         train_ds = [df[index] for index in train_index]

#         valid_ds = [df[index] for index in test_index]

#         label_pos , label_neg = count_lablel(train_ds)
#         print(f'train positive label: {label_pos} - train negative label: {label_neg}')

#         label_pos , label_neg = count_lablel(valid_ds)
#         print(f'Test positive label: {label_pos} - Test negative label: {label_neg}')

#         train = []
#         lbls_train = []
#         valid = []
#         lbls_valid = []

#         for i , data in enumerate(train_ds):
#             embbed, lbl, smiles = data
#             train.append(embbed[0])
#             lbls_train.append(lbl.tolist())

#         for i , data in enumerate(valid_ds):
#             embbed, lbl, smiles = data
#             valid.append(embbed[0])
#             lbls_valid.append(lbl.tolist())

#         l_train = np.array(train).reshape(-1,1024,1)
#         lbls_train = np.array(lbls_train)

#         l_valid = np.array(valid).reshape(-1,1024,1)
#         lbls_valid = np.array(lbls_valid)

#         # create neural network model
#         sequential_net = sequential_model()
#         history = History()
#         P = sequential_net.fit(train, lbls_train, epochs = Epoch_S, batch_size = 128, callbacks=[history])

#         for j in range(100):
#             C=1
#             Before = int(P.history['accuracy'][-1]*100)
#             for i in range(2,Epoch_S+1):
#                 if  int(P.history['accuracy'][-i]*100) == Before:
#                     C=C+1
#                 else:
#                     C=1
#                 Before=int(P.history['accuracy'][-i]*100)
#                 print(Before)
#             if C==Epoch_S:
#                 break
#             P = sequential_net.fit(train, lbls_train, epochs = Epoch_S, batch_size = 128, callbacks=[history])
#         print(j+1)

#         score  = sequential_net.evaluate(valid, lbls_valid, verbose=1)
#         a = (score[1],score[4])
#         result.append(a)

#         if score[4] > s :
#             best_model = sequential_net
#             s = score[4]
#             print("Save_model")


#     return result , best_model

# scores, best_model = evaluate_model(train, 10, True)